# go-Panic和Reccover

Go没有像Java那样的异常机制，它不能抛出异常，而是使用了panic和recover机制。

一定要记住:
* 你应当把它作为最后的手段来使用，也就是说，你的代码中应当没有，或者很少有panic的东西。
* 这是个强大的工具，请明智地使用它。

Go的类型系统会在编译时捕获很多错误，但有些错误只能在运行时检查，如数组访问越界、空指针引用等。这些运行时错误会引起painc异常。

不是所有的panic异常都来自运行时，直接调用内置的panic函数也会引发panic异常；

panic函数接受任何值作为参数。当某些不应该发生的场景发生时，我们就应该调用panic。比如，当程序到达了某条逻辑上不可能到达的路径：

断言函数必须满足的前置条件是明智的做法，但这很容易被滥用。除非你能提供更多的错误信息，或者能更快速的发现错误，否则不需要使用断言，编译器在运行时会帮你检查代码。

panic会引起程序的崩溃，因此panic一般用于严重错误，如程序内部的逻辑不一致

MustCompile不能接收不合法的输入。函数名中的Must前缀是一种针对此类函数的命名约定

为了方便诊断问题，runtime包允许程序员输出堆栈信息。



### Panic

* Panic是一个内建函数，可以中断原有的控制流程，进入一个panic状态中。
* 当函数F调用panic，函数F的执行被中断，但是F中的延迟函数会正常执行，然后F返回到调用它的地方。在调用的地方，F的行为就像调用了panic。这一过程继续向上，直到发生panic的goroutine中所有调用的函数返回，此时程序退出。
* panic可以直接调用panic产生。也可以由运行时错误产生，例如访问越界的数组。

```
var user = os.Getenv("USER")

func init() {
	if user == "" {
		panic("no value for $USER")
	}
}
```

### Recover

* Recover是一个内建的函数，可以让进入panic状态的goroutine恢复过来。
* recover仅在延迟函数中有效。在正常的执行过程中，调用recover会返回nil，并且没有其它任何效果。
* 如果当前的goroutine陷入panic状态，调用recover可以捕获到panic的输入值，并且恢复正常的执行。

```
func throwsPanic(f func()) (b bool) {
	defer func() {
		if x := recover(); x != nil {
			b = true
		}
	}()
	f() //执行函数f，如果f中出现了panic，那么就可以恢复回来
	return
}
```

如果在deferred函数中调用了内置函数recover，并且定义该defer语句的函数发生了panic异常，recover会使程序从panic中恢复，并返回panic value。导致panic异常的函数不会继续运行，但能正常返回。在未发生panic时调用recover，recover会返回nil。

不加区分的恢复所有的panic异常，不是可取的做法；因为在panic之后，无法保证包级变量的状态仍然和我们预期一致。比如，对数据结构的一次重要更新没有被完整完成、文件或者网络连接没有被关闭、获得的锁没有被释放。此外，如果写日志时产生的panic被不加区分的恢复，可能会导致漏洞被忽略。

有些情况下，我们无法恢复。某些致命错误会导致Go在运行时终止程序，如内存不足。